In [1]:
import pandas as pd
import json
import matplotlib.pyplot as plt
from obspy.clients.filesystem.tsindex import Client
from obspy import UTCDateTime
from tqdm import tqdm
import concurrent.futures
import numpy as np
import obspy
import h5py
from concurrent.futures import ProcessPoolExecutor, as_completed
from absl import logging

In [2]:
logging.set_verbosity(logging.INFO)

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
catalog_df=pd.read_csv("./data/TongaML_final_catalog.csv")
catalog_df["time"]=pd.to_datetime(catalog_df["time"])
catalog_df=catalog_df.drop(columns=["horizontal uncertainty (km)", "vertical uncertainty (km)", "origin time uncertainty (s)", "largest azimuthal gap (degree)", "ISCid"])
catalog_df.rename(columns={"latitude (degree)":"latitude", "longitude (degree)":"longitude", "depth (km)":"depth"}, inplace=True)
catalog_df.rename(columns={"id": "EVENT_ID", "latitude": "ELAT", "longitude": "ELON", "depth": "EDEP", "time": "ORIGIN_TIME"}, inplace=True)
print(len(catalog_df))

13616


In [ ]:
catalog_df.head()

,EVENT_ID,ELAT,ELON,EDEP,ORIGIN_TIME
0,49,-24.0547,-178.7527,581.54,2010-09-05 06:20:43.697
1,59,-22.9965,-175.6553,85.23,2010-02-03 07:50:12.697
2,64,-23.1014,-178.6428,554.76,2010-07-21 13:36:11.900
3,78,-21.8630,-175.5906,140.71,2010-02-07 15:23:13.122
4,95,-20.3652,-177.1233,446.39,2010-04-16 07:59:40.671


In [6]:
picks_df=pd.read_csv("./data/TongaML_final_picks.csv")
picks_df["time"]=pd.to_datetime(picks_df["time"])
picks_df["net"]=picks_df["id"].str.split(".").str[0]
picks_df["sta"]=picks_df["id"].str.split(".").str[1]
picks_df.drop(columns=["id"],inplace=True)
# time	type	event_index	net	sta
picks_df.rename(columns={"event_index": "EVENT_ID", "net": "NETWORK", "sta": "STATION"}, inplace=True)
print(len(picks_df))

417570


In [7]:
print(len(picks_df[picks_df["type"]=="P"]))
print(len(picks_df[picks_df["type"]=="S"]))
picks_df.head()

338812
78758


,time,type,EVENT_ID,NETWORK,STATION
0,2009-11-04 20:43:53.969900,P,98879,II,MSVF
1,2009-11-06 16:32:12.919900,P,79255,II,MSVF
2,2009-11-08 16:39:36.679900,P,98419,II,MSVF
3,2009-11-10 08:05:07.049900,P,71320,II,MSVF
4,2009-11-10 08:17:57.599900,P,14608,II,MSVF


In [8]:
def join_and_filter_dataframes(catalog_df, picks_df):
    """
    Joins catalog_df and picks_df, filters and selects specific columns,
    and identifies problematic (event_index, station) combinations.

    Args:
        catalog_df: DataFrame with EVENT_ID, ELAT, ELON, EDEP, ORIGIN_TIME.
        picks_df: DataFrame with time, type, EVENT_INDEX, NETWORK, STATION.

    Returns:
        A tuple containing:
        - result_df: The joined and filtered DataFrame.
        - problem_set: A set of problematic (event_index, station) combinations.
    """

    # Create PTIME and STIME columns in picks_df
    picks_df['PTIME'] = picks_df.apply(lambda row: row['time'] if row['type'] == 'P' else None, axis=1)
    picks_df['STIME'] = picks_df.apply(lambda row: row['time'] if row['type'] == 'S' else None, axis=1)

    # Group by EVENT_ID and STATION to find problematic combinations
    problem_set = set()
    
    def check_problems(group):
      p_count = group['PTIME'].notna().sum()
      s_count = group['STIME'].notna().sum()
      
      if p_count >= 2 or s_count >= 2 or (s_count > 0 and p_count == 0):
        problem_set.add((group['EVENT_ID'].iloc[0], group['STATION'].iloc[0]))

    picks_df.groupby(['EVENT_ID', 'STATION']).apply(check_problems)
    
    
    # Merge DataFrames with a left join on EVENT_ID, STATION and NETWORK
    # Use inner join to match the unique event in catalog_df and station/network in picks_df
    
    picks_df = picks_df.groupby(["EVENT_ID","STATION","NETWORK"]).agg(PTIME = ("PTIME", "min"), STIME = ("STIME", "min")).reset_index()

    merged_df = pd.merge(catalog_df, picks_df, on='EVENT_ID', how='left')

    # Select desired columns and rename them if needed
    result_df = merged_df[['EVENT_ID', 'ORIGIN_TIME', 'STATION', 'NETWORK', 'ELON', 'ELAT', 'EDEP', 'PTIME', 'STIME']]
    
    # Add SLON and SLAT (assuming they are needed later - they are not present in your sample dataframes)
    # You would likely need to join with another dataframe containing station locations here
    # For example, if you have a 'stations_df' with columns 'STATION', 'NETWORK', 'SLON', 'SLAT':
    # result_df = pd.merge(result_df, stations_df[['STATION', 'NETWORK', 'SLON', 'SLAT']], on=['STATION', 'NETWORK'], how='left')
    # For now, let's add placeholder columns
    result_df['SLON'] = None  # Placeholder
    result_df['SLAT'] = None  # Placeholder
    
    return result_df, problem_set

In [9]:
with open('./data/stations.json') as f:
    stations_raw = json.load(f)

stations={}
for key in stations_raw:
    sta=key.split('.')[1]
    stations[sta]={
        "STATION":sta,
        "latitude":stations_raw[key]['latitude'],
        "longitude":stations_raw[key]['longitude'],
        "elevation_m":stations_raw[key]['elevation_m'],
        "NETWORK":key.split('.')[0],
    }
stations_df=pd.DataFrame.from_dict(stations,orient="index")
print(len(stations_df))

67


In [10]:
stations_df.head()

,STATION,latitude,longitude,elevation_m,NETWORK
A01,A01,-21.533899,-175.623505,-1059.0,YL
A02W,A02W,-21.489500,-175.789993,-2015.0,YL
A03,A03,-21.443100,-175.896103,-1955.0,YL
A05,A05,-21.352100,-176.171005,-2368.0,YL
A06W,A06W,-21.303699,-176.321594,-2137.0,YL


In [11]:
# 1. Prepare picks_df
picks_pivot = picks_df.pivot_table(
    index=['EVENT_ID', 'NETWORK', 'STATION'],
    columns='type',
    values='time',
    aggfunc='first'
).reset_index()
picks_pivot = picks_pivot.rename(columns={'P': 'PTIME', 'S': 'STIME'})

# 2. Merge with stations_df to get station coordinates and SEVL.
picks_pivot = picks_pivot.merge(stations_df, on=['STATION', 'NETWORK'], how='left')
picks_pivot = picks_pivot.rename(
    columns={
        'latitude': 'SLAT',
        'longitude': 'SLON',
        'elevation_m': 'SEVL'
    }
)

# 3. Merge with catalog_df to get event details
merged_df = catalog_df.merge(picks_pivot, on='EVENT_ID', how='left')

# 4. Select and Reorder Columns
final_df = merged_df[[
    'EVENT_ID', 'ORIGIN_TIME', 'STATION', 'NETWORK', 'ELON', 'ELAT',
    'EDEP', 'PTIME', 'STIME', 'SLON', 'SLAT', 'SEVL'
]]

print(len(final_df))
print(len(final_df.dropna(subset=["PTIME"])))
print(len(final_df.dropna(subset=["STIME"])))
final_df.head(n=-10)

347829
338812
78758


,EVENT_ID,ORIGIN_TIME,STATION,NETWORK,ELON,ELAT,EDEP,PTIME,STIME,SLON,SLAT,SEVL
0,49,2010-09-05 06:20:43.697,A01,YL,-178.7527,-24.0547,581.54,2010-09-05 06:22:03.761000,NaT,-175.623505,-21.533899,-1059.0
1,49,2010-09-05 06:20:43.697,A02W,YL,-178.7527,-24.0547,581.54,2010-09-05 06:22:03.603700,NaT,-175.789993,-21.489500,-2015.0
2,49,2010-09-05 06:20:43.697,A03,YL,-178.7527,-24.0547,581.54,2010-09-05 06:22:03.945000,NaT,-175.896103,-21.443100,-1955.0
3,49,2010-09-05 06:20:43.697,A14W,YL,-178.7527,-24.0547,581.54,2010-09-05 06:22:00.615100,NaT,-178.155304,-20.665800,-884.0
4,49,2010-09-05 06:20:43.697,B03,YL,-178.7527,-24.0547,581.54,2010-09-05 06:22:15.740000,NaT,-175.385498,-19.941401,-2015.0
...,...,...,...,...,...,...,...,...,...,...,...,...
347814,109051,2010-06-08 01:00:19.543,C13,YL,-166.0604,-7.0963,136.66,2010-06-08 01:12:21.913000,NaT,-176.358093,-20.453600,-2280.0
347815,109051,2010-06-08 01:00:19.543,C15,YL,-166.0604,-7.0963,136.66,2010-06-08 01:03:03.296000,NaT,-176.715195,-20.048000,-2512.0
347816,109051,2010-06-08 01:00:19.543,C16W,YL,-166.0604,-7.0963,136.66,2010-06-08 01:07:52.477600,NaT,-176.828903,-20.570400,-2213.0
347817,109051,2010-06-08 01:00:19.543,F02W,YL,-166.0604,-7.0963,136.66,2010-06-08 01:06:16.150201,NaT,-174.438797,-21.349199,-2602.0


In [12]:
# only select final_df where PTIME exists
final_df=final_df.dropna(subset=["PTIME"])
print(len(final_df))

338812


In [13]:
final_df.to_csv("./result/TongaML_final_catalog_picks.csv",index=False)

## Prepare

In [14]:
client=Client("/mnt/scratch/xiziyi/database/timeseries.sqlite")

In [15]:
def process_row(row):
    problems = []
    start = UTCDateTime(row["PTIME"])-240
    end = UTCDateTime(row["PTIME"])+360
    net, sta, evid = row["NETWORK"], row["STATION"], row["EVENT_ID"]
    res = np.zeros((3,24000), dtype=np.float32)
    use_12z = False
    try:
        tr_z = client.get_waveforms(net, sta, "*", "*Z", start, end)[0]
        st_1 = client.get_waveforms(net, sta, "*", "*N", start, end)
        if len(st_1) == 0:
            tr_1 = client.get_waveforms(net, sta, "*", "*1", start, end)[0]
            use_12z = True
        else:
            tr_1 = st_1[0]
        st_2 = client.get_waveforms(net, sta, "*", "*E", start, end)
        if len(st_2) == 0:
            tr_2 = client.get_waveforms(net, sta, "*", "*2", start, end)[0]
            use_12z = True
        else:
            tr_2 = st_2[0]
        
        # prepare output
        d1=tr_1.data[:24000]
        d2=tr_2.data[:24000]
        dz=tr_z.data[:24000]
        res[0,:len(d1)]=d1[:]
        res[1,:len(d2)]=d2[:]
        res[2,:len(dz)]=dz[:]

        attrs = {
            "station": row["STATION"],
            "phase_type" :[],
            "phase_index": [],
            "component": ["1", "2", "Z"] if use_12z else ["N", "E", "Z"],
        }

        # if PTIME exists, add P phase type and index
        if not pd.isna(row["PTIME"]):
            attrs["phase_type"].append("P")
            attrs["phase_index"].append(int((UTCDateTime(row["PTIME"])-start)*40))
        # if STIME exists, add S phase type and index
        if not pd.isna(row["STIME"]):
            attrs["phase_type"].append("S")
            attrs["phase_index"].append(int((UTCDateTime(row["STIME"])-start)*40))
        return str(row["EVENT_ID"]), res, attrs, []

    except Exception as e:
        problems.append((net, sta, evid))
        return None, None, None, problems

In [16]:
def save_to_hdf5(event_id, data, attrs, h5py_file):
    if event_id not in h5py_file:
        h5py_file.create_group(event_id)
    group = h5py_file[event_id]
    wave = group.create_dataset(attrs["station"], data=data, compression="gzip", compression_opts=9)
    for key, value in attrs.items():
        wave.attrs[key] = value

In [17]:
process_df = final_df.copy()
num_cores = 50
h5py_file = h5py.File("./data_scratch/concurrency_files/TongaML_final.h5", "w")
submitting_bar = tqdm(total=len(process_df), desc="Submitting", position=0)
progress_bar = tqdm(total=len(process_df), desc="Processing", position=0)
with ProcessPoolExecutor(max_workers=num_cores) as executor:
    futures = []
    total_rows = len(process_df)
    for i in range(0, total_rows, num_cores):
        for index in range(num_cores):
            if i + index < total_rows:
                row = process_df.iloc[i + index]
                futures.append(executor.submit(process_row, row))
                submitting_bar.update(1)
    submitting_bar.close()

    for future in as_completed(futures):
        result = future.result()
        if result[0] is not None:
            save_to_hdf5(result[0], result[1], result[2], h5py_file)
        progress_bar.update(1)

progress_bar.close()
h5py_file.close()

Processing:   0%|          | 0/338812 [00:00<?, ?it/s]

Processing: 100%|██████████| 338812/338812 [1:07:41<00:00, 83.42it/s] 
